In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/emotion/emotion-twitter-lexicon.json

In [5]:
import tensorflow as tf
import numpy as np
from rotary_embedding_tensorflow import apply_rotary_emb, RotaryEmbedding
from fast_transformer import FastTransformer

In [6]:
from malaya.text.bpe import WordPieceTokenizer

In [7]:
tokenizer = WordPieceTokenizer('BERT.wordpiece', do_lower_case = False)
# tokenizer.tokenize('halo nama sayacomel')

In [8]:
import random
from unidecode import unidecode
import re

normalized_chars = {}

chars = '‒–―‐—━—-▬'
for char in chars:
    normalized_chars[ord(char)] = '-'

chars = '«»“”¨"'
for char in chars:
    normalized_chars[ord(char)] = '"'

chars = "’'ʻˈ´`′‘’\x92"
for char in chars:
    normalized_chars[ord(char)] = "'"

chars = '̲_'
for char in chars:
    normalized_chars[ord(char)] = '_'

chars = '\xad\x7f'
for char in chars:
    normalized_chars[ord(char)] = ''

chars = '\n\r\t\u200b\x96'
for char in chars:
    normalized_chars[ord(char)] = ' '

    
laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'hoho',
    'hewhew',
    'uwu',
    'sksk',
    'ksks',
    'gituu',
    'gitu',
    'mmeeooww',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday',
    'jahagaha',
    'ahakss',
    'ahksk'
}

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def cleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    
    chars = '.,/'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    x = []
    for word in string:
        word = word.lower()
        if any([laugh in word for laugh in laughing]):
            if random.random() >= 0.5:
                x.append(word)
        else:
            x.append(word)
    string = [w.title() if w[0].isupper() else w for w in x]
    return ' '.join(string)

In [9]:
import json
import random

emotion_label = ['anger', 'fear', 'happy', 'love', 'sadness', 'surprise']

with open('emotion-twitter-lexicon.json') as fopen:
    emotion = json.load(fopen)
    
emotion.keys()

dict_keys(['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'])

In [10]:
texts, labels = [], []

for k, v in emotion.items():
    if len(v) > 30000:
        emotion[k] = random.sample(v, 30000)
    print(k, len(emotion[k]))
    texts.extend(emotion[k])
    labels.extend([emotion_label.index(k)] * len(emotion[k]))

anger 30000
fear 20316
happy 30000
love 20783
sadness 26468
surprise 13107


In [11]:
from tqdm import tqdm

for i in tqdm(range(len(texts))):
    texts[i] = cleaning(texts[i])

100%|██████████| 140674/140674 [00:14<00:00, 9490.20it/s] 


In [12]:
actual_t, actual_l = [], []

for i in tqdm(range(len(texts))):
    if len(texts[i]) > 2:
        actual_t.append(texts[i])
        actual_l.append(labels[i])

100%|██████████| 140674/140674 [00:00<00:00, 871077.43it/s]


In [13]:
from tqdm import tqdm

input_ids, input_masks = [], []

for text in tqdm(actual_t):
    tokens_a = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    
    input_ids.append(input_id)
    input_masks.append(input_mask)

100%|██████████| 140674/140674 [00:55<00:00, 2531.72it/s]


In [14]:
epoch = 3
batch_size = 32
warmup_proportion = 0.1
num_train_steps = int(len(texts) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [15]:
import optimization

In [21]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        mask = tf.math.not_equal(self.X, 0)
        mask = tf.cast(mask, tf.bool)
        self.Y = tf.placeholder(tf.int32, [None])
        self.maxlen = tf.shape(self.X)[1]
        self.lengths = tf.count_nonzero(self.X, 1)
        
        self.model = FastTransformer(
            num_tokens = 32000,
            dim = 336,
            depth = 4,
            heads = 12,
            max_seq_len = 2048,
            absolute_pos_emb = True,
            mask = mask
        )
        self.logits = self.model(self.X)[0]
        self.logits_seq = tf.layers.dense(self.logits, dimension_output,
                                         kernel_initializer=create_initializer())
        
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [22]:
dimension_output = 6
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.trainable_variables()

Tensor("fast_transformer/pre_norm/fast_attention/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm/fast_attention/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_2/fast_attention_1/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_2/fast_attention_1/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_4/fast_attention_2/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_4/fast_attention_2/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_6/fast_attention_3/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_6/fast_attention_3/Select:0", shape=(?, 12, ?), dtype=float32)





In [23]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [24]:
tvars = tf.trainable_variables()
checkpoint = 'fastformer-tiny-social-media/model.ckpt-1000000'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [25]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from fastformer-tiny-social-media/model.ckpt-1000000


In [26]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [29]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_Y, test_Y, train_mask, test_mask = train_test_split(
    input_ids, actual_l, input_masks, test_size = 0.2
)

In [30]:
from tqdm import tqdm
import time

for EPOCH in range(epoch):

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_mask = train_mask[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_mask = test_mask[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
    
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 3/3517 [00:00<03:00, 19.51it/s, accuracy=1, cost=0.00916]

epoch: 0, training loss: 0.267515, training acc: 0.915987, valid loss: 0.100186, valid acc: 0.974731



train minibatch loop:   0%|          | 3/3517 [00:00<02:48, 20.90it/s, accuracy=1, cost=0.000318]

epoch: 1, training loss: 0.061159, training acc: 0.984502, valid loss: 0.066688, valid acc: 0.985207



test minibatch loop: 100%|██████████| 880/880 [00:21<00:00, 41.54it/s, accuracy=0.857, cost=0.609] 

epoch: 2, training loss: 0.033271, training acc: 0.991888, valid loss: 0.054080, valid acc: 0.988687



In [31]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'fastformer-tiny-emotion/model.ckpt')

'fastformer-tiny-emotion/model.ckpt'

In [32]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = test_Y[i: index]
    batch_mask = test_mask[i: index]
    batch_mask = pad_sequences(batch_mask, padding='post')
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 880/880 [00:19<00:00, 44.82it/s]


In [33]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'],
        digits = 5
    )
)

              precision    recall  f1-score   support

       anger    0.97568   0.99016   0.98287      5997
        fear    0.99179   0.98663   0.98920      4040
       happy    0.99570   0.99389   0.99479      6053
        love    0.98869   0.98301   0.98584      4179
     sadness    0.99293   0.99085   0.99189      5244
    surprise    0.99077   0.98246   0.98660      2622

    accuracy                        0.98880     28135
   macro avg    0.98926   0.98783   0.98853     28135
weighted avg    0.98885   0.98880   0.98881     28135



In [34]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'ReadVariableOp' not in n.name
        and 'AssignVariableOp' not in n.name
        and '/Assign' not in n.name
        and '/Adam' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'dense/kernel',
 'dense/bias',
 'logits_seq',
 'logits']

In [35]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [36]:
freeze_graph('fastformer-tiny-emotion', strings)

INFO:tensorflow:Restoring parameters from fastformer-tiny-emotion/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 58 variables.
INFO:tensorflow:Converted 58 variables to const ops.
2295 ops in the final graph.


In [37]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [38]:
# g = load_graph('fastformer-tiny-entities/frozen_model.pb')
# x = g.get_tensor_by_name('import/Placeholder:0')
# logits = g.get_tensor_by_name('import/logits:0')
# test_sess = tf.InteractiveSession(graph = g)

In [39]:
# %%time

# predicted = test_sess.run(logits,
#             feed_dict = {
#                 x: [parsed_sequence],
#             },
#     )[0]
# merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
# print(list(zip(merged[0], merged[1])))

In [40]:
from tensorflow.tools.graph_transforms import TransformGraph

In [42]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

input_nodes = [
    'Placeholder',
]
output_nodes = [
    'logits',
    'logits_seq'
]

pb = 'fastformer-tiny-emotion/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           input_nodes,
                                           output_nodes, transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

In [43]:
# g = load_graph('fastformer-tiny-entities/frozen_model.pb.quantized')
# x = g.get_tensor_by_name('import/Placeholder:0')
# logits = g.get_tensor_by_name('import/logits:0')
# test_sess = tf.InteractiveSession(graph = g)

In [44]:
# %%time

# predicted = test_sess.run(logits,
#             feed_dict = {
#                 x: [parsed_sequence],
#             },
#     )[0]
# merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
# print(list(zip(merged[0], merged[1])))

In [46]:
file = 'fastformer-tiny-emotion/frozen_model.pb'
outPutname = 'emotion/tiny-fastformer/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [47]:
file = 'fastformer-tiny-emotion/frozen_model.pb.quantized'
outPutname = 'emotion/tiny-fastformer-quantized/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)